In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
file_id = '1r4T5zff_e6ZDbVjsrBy_6AY1kf9RUJ67'
file_id2='16_CLJcTWC8pLyFvuZpoS2MsPRMFI7WQA'
file_id3='1PG5Iy6XvBYc7GD-eDFH_xW0xLOlx44Cf'
file_id4='17HwxICRmpOWydro3MezMh6-rzg0ZS86C'
downloaded = drive.CreateFile({'id': file_id4})
downloaded.GetContentFile('data.csv') 

import pandas as pd
df = pd.read_csv('data.csv') 
df.head()

,tag,starter_content
0,5,hi i have a coffee grinder that my employees a...
1,4,the fibaro fgk101 temperature doorwindow senso...
2,5,is there a way to pick and choose which sensor...
3,1,lg smartthinq is a published rest api lg devic...
4,2,just ordered the ge superbus 2000 automation m...


In [3]:
from fastai.text import *
from sklearn.model_selection import train_test_split

In [4]:
train, test = train_test_split(df, test_size=.2, random_state=2)
train, valid = train_test_split(train, test_size=.2, random_state=2)
print(len(train))
print(len(valid))
print(len(test))
print(train.head())

1378
345
431
      tag                                    starter_content
1103    1  i have some zwave ge wall dimmers and when i u...
1922    1  new smartthings app my devices show up under a...
953     1  i have a freezer in my garage which is all on ...
1517    5  i am trying to create an automation in the new...
534     1  i bought my first light switch ezex c2o model ...


In [5]:
!pip install pytorch-transformers
from pytorch_transformers import BertTokenizer, BertForSequenceClassification
bert_model = "bert-base-uncased"
max_seq_len = 128
batch_size = 32

In [6]:
bert_tokenizer = BertTokenizer.from_pretrained(bert_model)
bert_vocab = Vocab(list(bert_tokenizer.vocab.keys()))

In [7]:
class BertFastaiTokenizer(BaseTokenizer):
    def __init__(self, tokenizer, max_seq_len=128, **kwargs):
        self.pretrained_tokenizer = tokenizer
        self.max_seq_len = max_seq_len

    def __call__(self, *args, **kwargs):
        return self

    def tokenizer(self, t):
        return ["[CLS]"] + self.pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2] + ["[SEP]"]

In [8]:
tok_func = BertFastaiTokenizer(bert_tokenizer, max_seq_len=max_seq_len)
bert_fastai_tokenizer = Tokenizer(
    tok_func=tok_func,
    pre_rules = [],
    post_rules = []
)
path = Path(".")

In [9]:
databunch = TextClasDataBunch.from_df(path, train, valid, test,
                  tokenizer=bert_fastai_tokenizer,
                  vocab=bert_vocab,
                  include_bos=False,
                  include_eos=False,
                  text_cols="starter_content",
                  label_cols='tag',
                  bs=batch_size,
                  collate_fn=partial(pad_collate, pad_first=False, pad_idx=0),
             )
databunch.show_batch()

text,target
[CLS] last oct 2018 i purchased a connect home 3 pack from lowe ##s i was able to fire up 2 of the 3 hub ##s with no problem the power supply for the 3rd was no good lowe ##s didn ’ t have an individual one and wanted me to return the whole package of 3 that was unacceptable as i had my smoke detectors and sirens already connected,2
[CLS] this smart app and device type allows you to use any dl ##na media render ##er to receive son ##os ##like sound and voice notification ##s from smart ##thing ##s you must to install the media ##ren ##der ##er ##con ##ne ##ct to find any generic media render ##er network speaker the media ##ren ##der player is the device type needed to control each media render ##er if you,4
[CLS] so my hub which i purchased to replace another hub that died has also died no lights ##not ##hing power supply tests good contact samsung and they inform me since it is beyond the warrant ##y period that i have to purchase a new one it is beyond the warrant ##y period by three f ’ ing days that ’ s not the worst of it i am shocked,3
[CLS] kia ##no ##osh ##kara ##mi we believe this change is a reasonable and compromise between supporting old ##leg ##acy z ##wave devices and being z ##wave compliant the z ##wave specification requires that all generations be backwards ##com ##pati ##ble it always has the fiba ##ro and qu ##bino devices were certified under old generation specifications it is unfair and inaccurate to say that they are not compliant they,1
[CLS] this device type allows the integration of so ##m ##fy blinds operated via the so ##m ##fy my ##link hub it implements the smart ##thing ##s window shade and switch capabilities which means it will work with most automation ##s based on switching devices ono ##ff and will native ##ly work with any window shade integration ##s note that this is a wi ##p and there is still a,4


In [10]:
class MyNoTupleModel(BertForSequenceClassification):
  def forward(self, *args, **kwargs):
    return super().forward(*args, **kwargs)[0]

In [11]:
from collections import defaultdict
cate = defaultdict(int)
category=list(train.tag)
for i in range(len(train)):
    cate[str(category[i])] += 1
len(cate)#how many labels in train set

5

In [12]:
bert_pretrained_model = MyNoTupleModel.from_pretrained(bert_model, num_labels=5)

In [13]:
loss_func = nn.CrossEntropyLoss()

In [14]:
learn = Learner(databunch,
                bert_pretrained_model,
                loss_func=loss_func,
                metrics=accuracy)

In [17]:
learn.fit_one_cycle(2, 2e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.440963,1.383735,0.510145,1:33:24
1,1.355565,1.341260,0.510145,1:29:22
